In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import matplotlib.pyplot as plt
import json
import codecs
from tqdm import tqdm
import glob
import random
import argparse
import math
import time
import cv2

## Get the ground truth from ID

In [3]:
# load CSV file into NumPy array
resnet_prdiction_file = "/content/drive/Shareddrives/CIS519_Spring2023/train_data_prediction/resnet152b_predictions.csv"
megadetection_prediction_file = "/content/drive/Shareddrives/CIS519_Spring2023/train_data_prediction/megadetector_predictions.csv"
label_file = "/content/drive/Shareddrives/CIS519_Spring2023/train_data_prediction/labels.npy"
# read CSV file into Pandas DataFrame
resnet_df = pd.read_csv(resnet_prdiction_file)
mega_df = pd.read_csv(megadetection_prediction_file)
label_df = np.load(label_file)

In [12]:
resnet_df

,Id,Predicted
0,301cceec-7d42-11eb-8fb5-0242ac1c0002,1
1,300a44e8-7d42-11eb-8fb5-0242ac1c0002,2
2,302b1902-7d42-11eb-8fb5-0242ac1c0002,1
3,9936f868-21bc-11ea-a13a-137349068a90,2
4,300a242c-7d42-11eb-8fb5-0242ac1c0002,0
...,...,...
755,3021fcdc-7d42-11eb-8fb5-0242ac1c0002,2
756,2fffd65c-7d42-11eb-8fb5-0242ac1c0002,1
757,302bb614-7d42-11eb-8fb5-0242ac1c0002,3
758,300fdf20-7d42-11eb-8fb5-0242ac1c0002,1


In [5]:
mega_df.head()

,Id,Predicted
0,301cceec-7d42-11eb-8fb5-0242ac1c0002,3
1,300a44e8-7d42-11eb-8fb5-0242ac1c0002,4
2,302b1902-7d42-11eb-8fb5-0242ac1c0002,3
3,9936f868-21bc-11ea-a13a-137349068a90,2
4,300a242c-7d42-11eb-8fb5-0242ac1c0002,1


In [6]:
print(type(mega_df.iloc[0][1]))

<class 'numpy.int64'>


In [7]:
resnet_pred = np.zeros(len(resnet_df),dtype = np.int64)
mega_pred = np.zeros(len(resnet_df),dtype=np.int64)
for i in range(len(resnet_df)):
  resnet_pred[i] = resnet_df.iloc[i]["Predicted"]
  mega_pred[i] = mega_df.iloc[i]["Predicted"]

In [8]:
print(resnet_pred.shape)

(760,)


## Linear Regressor ensemble

In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
X = np.hstack((resnet_pred.reshape(-1,1), mega_pred.reshape(-1,1)))
y = label_df

In [11]:
print(X.shape)
print(y.shape)

(760, 2)
(760,)


In [13]:
X_train = X[0:500]
y_train = y[0:500]
X_test = X[500:-1]
y_test = y[500:-1]

In [14]:
model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

LinearRegression(fit_intercept=False)

In [15]:
ensemble_pred = model.predict(X_test)
loss = np.mean(np.abs(ensemble_pred - y_test))
print(loss)

1.0382161264640823


In [16]:
print(model.get_params())

{'copy_X': True, 'fit_intercept': False, 'n_jobs': None, 'positive': False}


In [17]:
model.coef_

array([0.36263452, 1.00057397])

## 